## Downloading Landsat images (mostly adapted from Quinn's 04_demo_download + stuff found online- glob)

In [ ]:
import pystac
from pystac_client import Client
import planetary_computer
import odc.stac
import matplotlib.pyplot as plt
import os
from pathlib import Path
import urllib
import rioxarray as rxr
import rasterio as rio
from glob import glob
import numpy as np

In [ ]:
#Set image directory 
imgdir = f'{Path.home()}/project/data'

if not os.path.exists(imgdir):
    os.makedirs(imgdir)

In [ ]:
#Collection and bands (specific bands for L2 surface reflectance/temperature products)
collection = 'landsat-c2-l2'
asset_id_list = ['SR_B1', 'SR_B2', 'SR_B3', 'SR_B4', 'SR_B5', 'SR_B6', 'ST_B10', "QA_PIXEL",'reduced_resolution_browse']
asset_id_list = ['coastal', 'blue', 'green', 'red', 'nir08', 'swir16', 'lwir11', 'qa_pixel','rendered_preview']

#Bounding box
bbox = [104.3,8.3,104.6,8.5]

#Date range
dt = '2026-01-01/2026-02-14'

catalog = Client.open("https://planetarycomputer.microsoft.com/api/stac/v1",modifier=planetary_computer.sign_inplace)

#Set path, row and cloud cover
results = catalog.search(
    collections=[collection],
    bbox=bbox,
    datetime=dt,
    query={
        "eo:cloud_cover": {"lt": 10},
        "landsat:wrs_path": {"eq": "126"}, # Path 126, Row 54 is the tile that contains Mui Ca Mau National Park.
        "landsat:wrs_row": {"eq": "054"},
    },
)

# Check how many items were returned
items = list(results.items())
print(f"Returned {len(items)} Items")

#Select 1 image to make the stacked image
item = items[0]

In [ ]:
#download bands to imgdir

for asset_id in asset_id_list:
        asset = item.assets[asset_id]
        print(asset.href)

        if asset_id == 'rendered_preview':
            out_fn = item.id + "_rendered_preview.png"
        else:
            out_fn = asset.href.split('?')[0].split('/')[-1]
            
        out_fp = os.path.join(imgdir, out_fn)

        # Check to see if file already exists
        if not os.path.exists(out_fp):
            print("Saving:", out_fp)
            # Download the file
            urllib.request.urlretrieve(asset.href, out_fp)

In [ ]:
#inspect data
!ls -lh $imgdir

In [ ]:
def process_landsat_l2(imgdir, year = '2026'):
    
    # Name band paths with glob
    band_paths = {
        "blue": glob(os.path.join(imgdir, "*SR_B2.TIF"))[0],
        "green": glob(os.path.join(imgdir, "*SR_B3.TIF"))[0],
        "red": glob(os.path.join(imgdir, "*SR_B4.TIF"))[0],
        "nir": glob(os.path.join(imgdir, "*SR_B5.TIF"))[0],
        "swir1": glob(os.path.join(imgdir, "*SR_B6.TIF"))[0],
        "qa": glob(os.path.join(imgdir, "*QA_PIXEL.TIF"))[0]
    }

     # Open blue band and save metadata
    with rio.open(band_paths["blue"]) as src:
        blue = src.read(1).astype(np.float32)
        meta = src.meta.copy()
    
    # Read bands the other bands
    def read_band(path, dtype=np.float32):
        with rio.open(path) as src:
            return src.read(1).astype(dtype)
    
    green = read_band(band_paths["green"])
    red = read_band(band_paths["red"])
    nir = read_band(band_paths["nir"])
    swir1 = read_band(band_paths["swir1"])
    qa = read_band(band_paths["qa"],dtype=np.uint16) #QA_PIXEL won't work if converted to float32
    
    # Convert to surfacer reflectance
    def scale_sr(band):
        return band * 0.0000275 - 0.2
    
    blue = scale_sr(blue)
    green = scale_sr(green)
    red = scale_sr(red)
    nir = scale_sr(nir)
    swir1 = scale_sr(swir1)
    
    # Mask clouds using QA_PIXEL band
    # Based on this guy's code- https://www.reddit.com/r/remotesensing/comments/thjhlf/landsat_8_cloud_mask_accuracy/#:~:text=My%20code%20for%20generating%20the,.astype(np.int16)
    # 3 = cloud
    # 4 = cloud shadow
    cloud = (qa & (1 << 3)) == 0
    
    mask = cloud 

    # Compute indices
    #NDVI = Normalized Difference Vegetation Index
    #NDWI = Normalized Difference Water Index
    #MVI = Mangrove Vegetation Index
    ndvi = (nir - red) / (nir + red)
    ndwi = (green - nir) / (green + nir)
    mvi  = (nir - green) / (swir1 - green)

    
    def apply_mask(band):
        band[~mask] = np.nan
        return band
    
    blue = apply_mask(blue)
    green = apply_mask(green)
    red = apply_mask(red)
    nir = apply_mask(nir)
    swir1 = apply_mask(swir1)
    ndvi = apply_mask(ndvi)
    ndwi = apply_mask(ndwi)
    mvi = apply_mask(mvi)
    
    # Normalize bands/indices to stack
    def normalize(band):
        return (band - np.nanmin(band)) / (np.nanmax(band) - np.nanmin(band))
    
    red_n = normalize(red)
    nir_n = normalize(nir)
    swir_n = normalize(swir1)
    ndvi_n = normalize(ndvi)
    ndwi_n = normalize(ndwi)
    mvi_n = normalize(mvi)

    #Save normalized images- except for the indices, just save those as normal (normalized used for stacking)
    meta.update(dtype="float32", nodata=np.nan)
    
    def save_raster(array, name):
        out_path = os.path.join(imgdir, f"{year}_{name}.tif")
        with rio.open(out_path, "w", **meta) as dst:
            dst.write(array.astype("float32"), 1)
    
    save_raster(red_n, "red_norm")
    save_raster(nir_n, "nir_norm")
    save_raster(swir_n, "swir1_norm")
    save_raster(ndvi, "ndvi")
    save_raster(ndwi, "ndwi")
    save_raster(mvi, "mvi")
    
    # Stack images
    stack = np.dstack([
        red_n,
        nir_n,
        swir_n,
        ndvi_n,
        ndwi_n,
        mvi_n
    ])

       # Update metadata for multi-band output
    stack_meta = meta.copy()
    stack_meta.update(count=stack.shape[2])
    
    stack_path = os.path.join(imgdir, f"{year}_segmentation_stack.tif")
    
    with rio.open(stack_path, "w", **stack_meta) as dst:
        for i in range(stack.shape[2]):
            dst.write(stack[:, :, i].astype("float32"), i+1)
            
    
    return stack, red, nir, ndvi

In [ ]:
#Run for this year
process_landsat_l2(imgdir,'2026')